In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [5]:


# Parameters
learning_rate = 0.01
training_epochs = 5
batch_size = 256

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)

X = tf.placeholder(tf.float32, shape=(None, n_input))
X_noise = tf.placeholder(tf.float32, shape=(None, n_input))

AttributeError: module 'tensorflow' has no attribute 'Placeholder'

In [ ]:
weights = {
    'encoder_w1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
    'encoder_w2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.1)),
    'decoder_w1': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_1], stddev=0.1)),
    'decoder_w2': tf.Variable(tf.truncated_normal([n_hidden_1, n_input], stddev=0.1))
}

bias = {
    "encoder_b1": tf.Variable(tf.truncated_normal([1, n_hidden_1], stddev=0.1)),
    "encoder_b2": tf.Variable(tf.truncated_normal([1, n_hidden_2], stddev=0.1)),
    "decoder_b1": tf.Variable(tf.truncated_normal([1, n_hidden_1], stddev=0.1)),
    "decoder_b2": tf.Variable(tf.truncated_normal([1, n_input], stddev=0.1))
}

def encoder(X):
    layer1 = tf.nn.sigmoid(tf.matmul(X, weights['encoder_w1']) + bias['encoder_b1'])
    layer2 = tf.nn.sigmoid(tf.matmul(layer1, weights['encoder_w2']) + bias['encoder_b2'])
    return layer2
    
def decoder(x):
    layer1 = tf.nn.sigmoid(tf.matmul(x, weights['decoder_w1']) + bias['decoder_b1'])
    layer2 = tf.nn.sigmoid(tf.matmul(layer1, weights['decoder_w2']) + bias['decoder_b2'])
    return layer2

In [8]:
mypath = "./noise_generation/tmp"
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
num_of_pictures = 150

 

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
original_images = np.empty([num_of_pictures,256,256,3])
noised_images = np.empty([num_of_pictures,256,256,3])

 

iteration = 0
countOdd = 0
countEven = 0

 

for q in onlyfiles[:num_of_pictures]:
    name = mypath+"/"+q
   # images.append(cv2.imread(name))
   
    if (iteration % 2 == 0):
        noised_images[countOdd] = image.img_to_array(image.load_img(name, target_size=(256, 256)))
        noised_images[countOdd] /= 255.0
        countOdd = countOdd + 1    
    else:
        original_images[countEven] = image.img_to_array(image.load_img(name, target_size=(256, 256)))
        #original_images[countEven] = cv2.imread(name)
        original_images[countEven] /= 255.0
        countEven = countEven + 1
    iteration += 1
    
print(original_images[1][1][1])
split = int(original_images.size*0.7)

 

train_orig = original_images[:split]
train_noised = noised_images[:split]
                        
test_orig = original_images[split:]
test_noised = noised_images[split:]

[0.18823529 0.30980392 0.56470588]


In [9]:
batch_size = 100
IMG_SHAPE = 150

In [10]:
model = Sequential()

model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

In [11]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
epochs = 80

history = model.fit_generator(
  train_noised,
    steps_per_epoch=int(np.ceil(train_noised.size / float(batch_size))),
    epochs=epochs,
    validation_data=train_orig,
    validation_steps=int(np.ceil(train_orig.size / float(batch_size)))
)

ValueError: When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.